In [ ]:
!pip install fastFM==0.2.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 259 kB 6.0 MB/s 
  Created wheel for fastFM: filename=fastFM-0.2.9-cp37-cp37m-linux_x86_64.whl size=432612 sha256=ad0d2b69b9980cb55590050303c38f8a7deb8f8aee2665aa551d08c55c43b8a6
  Stored in directory: /root/.cache/pip/wheels/73/64/93/8ac211b2001b1befac04bb1579dcf7bdd583afef4fd41c1a39
Successfully built fastFM


In [ ]:
# !pip list | grep fastFM

In [ ]:
# !apt-get install python-dev libopenblas-dev
# !git clone --recursive https://github.com/ibayer/fastFM.git
# import os
# os.chdir("/content/fastFM/")
# !pip install -r ./requirements.txt
# !make
# !PYTHON=python3 make
# !pip install .

In [ ]:
import os, numpy as np
path = "/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/working_folder"
os.chdir(path)

In [ ]:
import gzip
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy
from collections import defaultdict
from fastFM import als
from scipy.spatial import distance
import csv
import dateutil.parser
from datetime import timedelta
import pickle

In [ ]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [ ]:
# with open("Cat_price_filtered_review_train.pkl","rb") as fp:
#      all_dicts = pickle.load(fp)

# with open("Cat_price_filtered_review_test.pkl","rb") as fp:
#      test_dicts = pickle.load(fp)

with open("Sampled_by_review_count_train_state.pkl","rb") as fp:
     train_dicts = pickle.load(fp)

with open("Sampled_by_review_count_val_state.pkl","rb") as fp:
     val_dicts = pickle.load(fp)

with open("Sampled_by_review_count_test_state.pkl","rb") as fp:
     test_dicts = pickle.load(fp)

In [ ]:
# len(all_dicts)

749747

In [ ]:
len(all_dicts)

383133

In [ ]:
# train_dicts = all_dicts[:700000]
# test_dicts = all_dicts[700000:]

In [ ]:
# val_dicts = all_dicts[600000:]
# train_dicts = all_dicts[:600000]

In [ ]:
# with open("Cat_price_filtered_review_test.pkl","wb") as fp:
#      pickle.dump(test_dicts,fp)

In [ ]:
# with open("Cat_price_filtered_review_train.pkl","wb") as fp:
#      pickle.dump(train_dicts,fp)

In [ ]:
with open("Price_filtered_places.pkl","rb") as fp:
     filtered_places = pickle.load(fp)

In [ ]:
#Places mapping from place id to its attributes

places_meta_data = defaultdict(dict)
for d in filtered_places:
  places_meta_data[d['gPlusPlaceId']] = d

In [ ]:
train_dicts[0]

{'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'gPlusUserId': '100000032416892623125',
 'gps': (40.179159, -122.236162),
 'parsed_time': 0.998995623918814,
 'rating': 4,
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'reviewTime': 'Mar 12, 2014',
 'reviewerName': 'william spindler',
 'state': 'CA',
 'unixReviewTime': 1394669496}

In [ ]:
users = set([x['gPlusUserId'] for x in train_dicts])
nUsers = len(users)
places = set([x['gPlusPlaceId'] for x in train_dicts])
nPlaces = len(places)

prices = set([x['price'] for x in filtered_places])
nPrices = len(prices)

user_di = {user:i for i,user in enumerate(list(users))}
place_di = {place:i for i,place in enumerate(list(places))}
price_di = {price:i for i,price in enumerate(list(prices))}

In [ ]:
X = scipy.sparse.lil_matrix((len(all_dicts), nUsers + nPlaces + nPrices))

for i in range(len(all_dicts)):
    user = user_di[all_dicts[i]['gPlusUserId']]
    item = place_di[all_dicts[i]['gPlusPlaceId']]
    price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X[i,user] = 1 # One-hot encoding of user
    X[i,nUsers + item] = 1 # One-hot encoding of item
    X[i,nUsers + nPlaces+price] = 1

y = numpy.array([d['rating'] for d in all_dicts])

# split = int(0.8*len(all_dicts))
# X_train,y_train = X[:split],y[:split]
# X_val,y_val = X[split:],y[split:]

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces + nPrices))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']

  place_id = test_dicts[i]['gPlusPlaceId']

  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]

    price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    X_test[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm = als.FMRegression(n_iter=500, init_stdev=0.1, rank=3, l2_reg_w=20, l2_reg_V=50)

fm.fit(X_train, y_train)

In [ ]:
y_pred_train = fm.predict(X_train)
MSE(y_pred_train, y_train)

y_pred_val = fm.predict(X_val)
MSE(y_pred_val, y_val)

y_test_pred = fm.predict(X_test)

1.1778189713842855

In [ ]:
sorted_review_count = sorted(review_count_per_user,reverse=True)

In [ ]:
sum(sorted_review_count[:25000])

216467

In [ ]:
# for i in sorted_review_count:
#   if i == 2:
#     break
print(sorted_review_count.index(1))

81990


In [ ]:
sum(sorted_review_count[:40000]) + 80000+ 40000

381686

In [ ]:
#Sampling scheme

users_with_non1_rcount = [user for user in placesPerUser if len(placesPerUser[user]) >=2 ]
users_with_1_rcount = [user for user in placesPerUser if len(placesPerUser[user]) ==1 ]
users_with_1_rcount_subset = [random.choice(users_with_1_rcount) for _ in range(40000)]
users_with_non1_rcount.extend(users_with_1_rcount_subset)
user_list_sampled = set(users_with_non1_rcount)
sampled_train_dict = [d for d in train_dicts if d['gPlusUserId'] in user_list_sampled]

print(len(user_list_sampled))
print(len(sampled_train_dict))

119022
383133


In [ ]:
# with open("Sampled_by_review_count_train.pkl","wb") as fp:
#      pickle.dump(sampled_train_dict,fp)

In [ ]:
# train_dicts = sampled_train_dict

In [ ]:
time_li = [d['unixReviewTime'] for d in train_dicts]
max_time = max(time_li)

users = set([x['gPlusUserId'] for x in train_dicts])
nUsers = len(users)
places = set([x['gPlusPlaceId'] for x in train_dicts])
nPlaces = len(places)

prices = set([x['price'] for x in filtered_places])
nPrices = len(prices)

user_di = {user:i for i,user in enumerate(list(users))}
place_di = {place:i for i,place in enumerate(list(places))}
price_di = {price:i for i,price in enumerate(list(prices))}

In [ ]:
mod_train_dicts = []
allRatings = []
ratingDict = {}
userRatings = defaultdict(list) #all ratings given by user u
placeRatings = defaultdict(list)
usersPerPlace = defaultdict(set) # Maps an item to the users who rated it
placesPerUser = defaultdict(set) # Maps a user to the items that they rated

for d in train_dicts:
  user = d['gPlusUserId']
  place = d['gPlusPlaceId']
  usersPerPlace[place].add(user)
  placesPerUser[user].add(place)
  r = int(d['rating'])
  d['rating'] = r
  ratingDict[(user,place)] = r
  d['parsed_time'] = d['unixReviewTime']/max_time
  # for key in recipe_meta_data[place]:
  #   d[key] = recipe_meta_data[place][key]
  allRatings.append(r)
  userRatings[user].append((r,d['parsed_time']))
  placeRatings[place].append((r,d['parsed_time']))
  mod_train_dicts.append(d)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum([t[0] for t in userRatings[u]]) / len(userRatings[u])

In [ ]:
review_count_per_user = [len(placesPerUser[user]) for user in placesPerUser]
avg_review_count = sum(review_count_per_user)/len(review_count_per_user)
avg_review_count

3.219009930937138

In [ ]:
y_val_glob_avg = [globalAverage]*len(y_val)

In [ ]:
MSE(y_val,y_val_glob_avg)

1.46733254963012

Vanilla FM

In [ ]:
X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])

# split = int(0.8*len(all_dicts))
# X_train,y_train = X[:split],y[:split]
# X_val,y_val = X[split:],y[split:]

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']

  place_id = val_dicts[i]['gPlusPlaceId']

  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']

  place_id = test_dicts[i]['gPlusPlaceId']

  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=20)

fm.fit(X_train, y_train)

FMRegression(l2_reg_w=20, rank=0)

In [ ]:
y_pred_train = fm.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm.predict(X_test)

1.0264648031272718
1.4136195046086115


FastFM with state feature

In [ ]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",""]

state_coords = [(32.806671,-86.79113),	(61.370716,-152.404419),	(33.729759,-111.431221),	(34.969704,-92.373123),	(36.116203,-119.681564),	(39.059811,-105.311104),	(41.597782,-72.755371),	(39.318523,-75.507141),	(27.766279,-81.686783),	(33.040619,-83.643074),	(21.094318,-157.498337),	(44.240459,-114.478828),	(40.349457,-88.986137),	(39.849426,-86.258278),	(42.011539,-93.210526),	(38.5266,-96.726486),	(37.66814,-84.670067),	(31.169546,-91.867805),	(44.693947,-69.381927),	(39.063946,-76.802101),	(42.230171,-71.530106),	(43.326618,-84.536095),	(45.694454,-93.900192),	(32.741646,-89.678696),	(38.456085,-92.288368),	(46.921925,-110.454353),	(41.12537,-98.268082),	(38.313515,-117.055374),	(43.452492,-71.563896),	(40.298904,-74.521011),	(34.840515,-106.248482),	(42.165726,-74.948051),	(35.630066,-79.806419),	(47.528912,-99.784012),	(40.388783,-82.764915),	(35.565342,-96.928917),	(44.572021,-122.070938),	(40.590752,-77.209755),	(41.680893,-71.51178),	(33.856892,-80.945007),	(44.299782,-99.438828),	(35.747845,-86.692345),	(31.054487,-97.563461),	(40.150032,-111.862434),	(44.045876,-72.710686),	(37.769337,-78.169968),	(47.400902,-121.490494),	(38.491226,-80.954453),	(44.268543,-89.616508),	(42.755966,-107.30249), (0.0,0.0)]

In [ ]:
nStates = 51
X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+nStates))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    X_train[i,nUsers + nPlaces + state] = 1
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])

# split = int(0.8*len(all_dicts))
# X_train,y_train = X[:split],y[:split]
# X_val,y_val = X[split:],y[split:]

In [ ]:
# temp = X_train[0].todense()

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+nStates))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']

  place_id = val_dicts[i]['gPlusPlaceId']
  state = state_list.index(val_dicts[i]['state'])
  X_val[i,nUsers + nPlaces + state] = 1
  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces + state] = 1
    #X_val[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces+nStates))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']

  place_id = test_dicts[i]['gPlusPlaceId']
  state = state_list.index(test_dicts[i]['state'])
  X_test[i,nUsers + nPlaces + state] = 1
  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm1 = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=20)

fm1.fit(X_train, y_train)

FMRegression(l2_reg_w=20, rank=0)

In [ ]:
y_pred_train = fm1.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm1.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm1.predict(X_test)

1.025099194760506
1.4119281713148428


FISM with just places

In [ ]:
X_train_fism = scipy.sparse.lil_matrix((len(train_dicts), nPlaces*2))

for i in range(len(train_dicts)):
    #user = user_di[train_dicts[i]['gPlusUserId']]
    user_id = train_dicts[i]['gPlusUserId']
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_train_fism[i,item] = 1 # One-hot encoding of user
    for item_id in placesPerUser[user_id]:
      item_query = place_di[item_id]
      if item != item_query:
        X_train_fism[i,nPlaces + item_query] = 1/(len(placesPerUser[user_id]) - 1) # One-hot encoding of item

    #X_train_fism[i,nUsers + nPlaces+price] = 1

y_train_fism = numpy.array([d['rating'] for d in train_dicts])

In [ ]:
X_val_fism = scipy.sparse.lil_matrix((len(val_dicts), nPlaces*2))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']

  place_id = val_dicts[i]['gPlusPlaceId']

  if(user_id in user_di and  place_id in place_di):
    user_id = val_dicts[i]['gPlusUserId']
    item = place_di[val_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val_fism[i,item] = 1 # One-hot encoding of user
    for item_id in placesPerUser[user_id]:
      item_query = place_di[item_id]
      if item != item_query:
        X_val_fism[i,nPlaces + item_query] = 1/(len(placesPerUser[user_id]) - 1)

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    #X_val_fism[i,nUsers + nPlaces+price] = 1

  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val_fism[i,item] = 1 # One-hot encoding of item
    #X_val_fism[i,nUsers + nPlaces+price] = 1

y_val_fism = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test_fism = scipy.sparse.lil_matrix((len(test_dicts), nPlaces*2))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']

  place_id = test_dicts[i]['gPlusPlaceId']

  if(user_id in user_di and  place_id in place_di):
    user_id = test_dicts[i]['gPlusUserId']
    item = place_di[test_dicts[i]['gPlusPlaceId']]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test_fism[i,item] = 1 # One-hot encoding of user
    for item_id in placesPerUser[user_id]:
      item_query = place_di[item_id]
      if item != item_query:
        X_test_fism[i,nPlaces + item_query] = 1/(len(placesPerUser[user_id]) - 1)

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    #X_val_fism[i,nUsers + nPlaces+price] = 1

  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test_fism[i,item] = 1 # One-hot encoding of item
    #X_val_fism[i,nUsers + nPlaces+price] = 1

y_test_fism = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm_fism = als.FMRegression(init_stdev=0.1, rank=5, l2_reg_w=20, l2_reg_V=10)

fm_fism.fit(X_train_fism, y_train_fism)

y_pred_train = fm_fism.predict(X_train_fism)
print(MSE(y_pred_train, y_train_fism))

y_pred_val = fm_fism.predict(X_val_fism)
print(MSE(y_pred_val, y_val_fism))

y_test_pred = fm_fism.predict(X_test_fism)

1.1200668548540595
1.4124207094020438


FISm with state co-ords

In [ ]:

X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+2))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    state = state_list.index(train_dicts[i]['state'])
    cord = state_coords[state]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    X_train[i,nUsers + nPlaces + 0] = cord[0]
    X_train[i,nUsers + nPlaces + 1] = cord[1]
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+2))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']

  place_id = val_dicts[i]['gPlusPlaceId']
  state = state_list.index(val_dicts[i]['state'])
  cord = state_coords[state]
  X_val[i,nUsers + nPlaces + 0] = cord[0]
  X_val[i,nUsers + nPlaces + 1] = cord[1]
  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces + state] = 1
    #X_val[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces+2))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']

  place_id = test_dicts[i]['gPlusPlaceId']
  state = state_list.index(test_dicts[i]['state'])
  cord = state_coords[state]
  X_test[i,nUsers + nPlaces + 0] = cord[0]
  X_test[i,nUsers + nPlaces + 1] = cord[1]
  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm2 = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=20)

fm2.fit(X_train, y_train)

y_pred_train = fm2.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm2.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm2.predict(X_test)

1.0282877218789226
1.4168570405719452


#Fast FM with popularity of place and state

In [ ]:
placesPerState = defaultdict(int)

for d in train_dicts:
  placesPerState[d['state']] += 1

placesPerState[""] = 0

In [ ]:
X_train = scipy.sparse.lil_matrix((len(train_dicts), nUsers + nPlaces+2))

for i in range(len(train_dicts)):
    user = user_di[train_dicts[i]['gPlusUserId']]
    item = place_di[train_dicts[i]['gPlusPlaceId']]
    state = state_list.index(train_dicts[i]['state'])
    cord = state_coords[state]
    #price = price_di[places_meta_data[train_dicts[i]['gPlusPlaceId']]['price']]
    X_train[i,user] = 1 # One-hot encoding of user
    X_train[i,nUsers + item] = 1 # One-hot encoding of item
    X_train[i,nUsers + nPlaces + 0] = len(usersPerPlace[train_dicts[i]['gPlusPlaceId']])
    X_train[i,nUsers + nPlaces + 1] = placesPerState[train_dicts[i]['state']]
    #X_train[i,nUsers + nPlaces+price] = 1

y_train = numpy.array([d['rating'] for d in train_dicts])




In [ ]:
t = X_train[0].todense()

In [ ]:
t.nonzero()

(array([0, 0, 0]), array([ 96429, 159313, 237192]))

In [ ]:
len(usersPerPlace[40291])

0

In [ ]:
place_di[train_dicts[0]['gPlusPlaceId']]

40291

In [ ]:
X_val = scipy.sparse.lil_matrix((len(val_dicts), nUsers + nPlaces+2))

for i in range(len(val_dicts)):
  user_id = val_dicts[i]['gPlusUserId']

  place_id = val_dicts[i]['gPlusPlaceId']
  state = state_list.index(val_dicts[i]['state'])
  cord = state_coords[state]
  X_val[i,nUsers + nPlaces + 0] = len(usersPerPlace[place_id])
  X_val[i,nUsers + nPlaces + 1] = placesPerState[val_dicts[i]['state']]
  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]
    state = state_list.index(train_dicts[i]['state'])
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_val[i,user] = 1 # One-hot encoding of user
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_train[i,nUsers + nPlaces + state] = 1
    #X_val[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_val[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_val[i,nUsers + item] = 1 # One-hot encoding of item
    #X_val[i,nUsers + nPlaces+price] = 1

y_val = numpy.array([d['rating'] for d in val_dicts])

In [ ]:
X_test = scipy.sparse.lil_matrix((len(test_dicts), nUsers + nPlaces+2))

for i in range(len(test_dicts)):
  user_id = test_dicts[i]['gPlusUserId']

  place_id = test_dicts[i]['gPlusPlaceId']
  state = state_list.index(test_dicts[i]['state'])
  cord = state_coords[state]
  X_test[i,nUsers + nPlaces + 0] = len(usersPerPlace[place_id])
  X_test[i,nUsers + nPlaces + 1] = placesPerState[test_dicts[i]['state']]
  if(user_id in user_di and  place_id in place_di):

    user = user_di[user_id]
    item = place_di[place_id]

    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]

    X_test[i,user] = 1 # One-hot encoding of user
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

  elif(user_id in user_di):
    user = user_di[user_id]
    X_test[i,user] = 1 # One-hot encoding of user
  elif(place_id in place_di):
    item = place_di[place_id]
    #price = price_di[places_meta_data[all_dicts[i]['gPlusPlaceId']]['price']]
    X_test[i,nUsers + item] = 1 # One-hot encoding of item
    #X_test[i,nUsers + nPlaces+price] = 1

y_test = numpy.array([d['rating'] for d in test_dicts])

In [ ]:
fm3 = als.FMRegression(init_stdev=0.1, rank=0, l2_reg_w=5)

fm3.fit(X_train, y_train)

y_pred_train = fm3.predict(X_train)
print(MSE(y_pred_train, y_train))

y_pred_val = fm3.predict(X_val)
print(MSE(y_pred_val, y_val))

y_test_pred = fm3.predict(X_test)

0.7270189332824109
1.378270040682252


In [ ]:
print(MSE(y_test_pred, y_test))

1.3880269067845095
